Paper's Title Generation from Abstracts

# ***Installing the dependencies***

In [ ]:
!pip install datasets

     |████████████████████████████████| 311 kB 5.0 MB/s 
     |████████████████████████████████| 133 kB 50.1 MB/s 
     |████████████████████████████████| 1.1 MB 53.1 MB/s 
     |████████████████████████████████| 243 kB 67.5 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 66.2 MB/s 
     |████████████████████████████████| 271 kB 58.7 MB/s 


In [ ]:
!pip install simplet5

     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 2.8 MB 45.3 MB/s 
     |████████████████████████████████| 919 kB 58.9 MB/s 
     |████████████████████████████████| 596 kB 60.5 MB/s 
     |████████████████████████████████| 396 kB 65.1 MB/s 
     |████████████████████████████████| 829 kB 56.6 MB/s 
     |████████████████████████████████| 3.3 MB 41.4 MB/s 
     |████████████████████████████████| 895 kB 51.5 MB/s 
  Created wheel for simplet5: filename=simplet5-0.1.3-py3-none-any.whl size=6832 sha256=bd560af93633700021898356575fcd1ae36b93d16d86d3013e7ae6e91ba5b46c
  Stored in directory: /root/.cache/pip/wheels/fb/87/9d/190f8091eed71a654458ca3a285743ed9ffeb55c2ffa4be68f
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=688643a99b147422ed64db7c11eceed928d64af76d2d0fd55b8b51ccc7956cd9
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built

In [ ]:
!pip install kaggle

# ***Reading the dataframe***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import dask.bag as db
import json
import pandas as pd

docs = db.read_text('/content/drive/MyDrive/Blog_3/arxiv-metadata-oai-snapshot.json').map(json.loads)

In [ ]:
# The dataset is very huge. Not sure if the whole set can be used. I start prototyping with a subset of the data so it's easyer to handel:
# This procedure was recommended in the ArXiv dataset itself

get_latest_version = lambda x: x['versions'][-1]['created']


# get only necessary fields of the metadata file
trim = lambda x: {'id': x['id'],
                  'authors': x['authors'],
                  'title': x['title'],
                  'doi': x['doi'],
                  'category':x['categories'].split(' '),
                  'abstract':x['abstract'],}
# filter for papers published on or after 2019-01-01
columns = ['id','category','abstract']
docs_df = (docs.filter(lambda x: int(get_latest_version(x).split(' ')[3]) > 2018)
           .map(trim).
           compute())

# convert to pandas
docs_df = pd.DataFrame(docs_df)

In [ ]:
docs_df

,id,authors,title,doi,category,abstract
0,0704.0479,T.Geisser,The affine part of the Picard scheme,None,"[math.AG, math.KT]",We describe the maximal torus and maximal un...
1,0704.1445,Yasha Gindikin and Vladimir A. Sablikov,Deformed Wigner crystal in a one-dimensional q...,10.1103/PhysRevB.76.045122,"[cond-mat.str-el, cond-mat.mes-hall]",The spatial Fourier spectrum of the electron...
2,0705.0033,"Nikos Frantzikinakis, Randall McCutcheon",Ergodic Theory: Recurrence,None,[math.DS],We survey the impact of the Poincar\'e recur...
3,0705.0344,J. P. Pridham,Unifying derived deformation theories,None,[math.AG],We develop a framework for derived deformati...
4,0705.0825,Ram Gopal Vishwakarma (Zacatecas University),Einstein's Theory of Gravity in the Presence o...,10.1007/s10509-009-0016-8,"[gr-qc, astro-ph, hep-th]",The mysterious `dark energy' needed to expla...
...,...,...,...,...,...,...
562117,quant-ph/0612050,"Igor Devetak, Jon Yard",The exact cost of redistributing multipartite ...,10.1103/PhysRevLett.100.230501,[quant-ph],How correlated are two quantum systems from ...
562118,quant-ph/0701163,Daegene Song,Does Observation Create Reality?,None,[quant-ph],It has been suggested that the locality of i...
562119,quant-ph/0702160,"Andrew M. Childs, Richard Cleve, Stephen P. Jo...",Discrete-query quantum algorithm for NAND trees,10.4086/toc.2009.v005a005,[quant-ph],"Recently, Farhi, Goldstone, and Gutmann gave..."
562120,quant-ph/9606017,Arthur Jabs,Quantum Mechanics in Terms of Realism,None,[quant-ph],.We expound an alternative to the Copenhagen...


In [ ]:
 docs_df = docs_df.drop(["id","authors","doi","category"],axis = 1)

In [ ]:
docs_df

,title,abstract
0,The affine part of the Picard scheme,We describe the maximal torus and maximal un...
1,Deformed Wigner crystal in a one-dimensional q...,The spatial Fourier spectrum of the electron...
2,Ergodic Theory: Recurrence,We survey the impact of the Poincar\'e recur...
3,Unifying derived deformation theories,We develop a framework for derived deformati...
4,Einstein's Theory of Gravity in the Presence o...,The mysterious `dark energy' needed to expla...
...,...,...
562117,The exact cost of redistributing multipartite ...,How correlated are two quantum systems from ...
562118,Does Observation Create Reality?,It has been suggested that the locality of i...
562119,Discrete-query quantum algorithm for NAND trees,"Recently, Farhi, Goldstone, and Gutmann gave..."
562120,Quantum Mechanics in Terms of Realism,.We expound an alternative to the Copenhagen...


In [ ]:
docs_df = docs_df.drop_duplicates()

In [ ]:
docs_df = docs_df.rename(columns = {'title': 'target_text', 'abstract': 'source_text'})

In [ ]:
# split the data into training and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(docs_df, test_size=0.2)

In [ ]:
train_df

,target_text,source_text
210743,Classification of discrete modular symmetries ...,We classify discrete modular symmetries in t...
276787,Vortex lattice in two-dimensional chiral XY fe...,"In this Letter we will show that, in the pre..."
205026,Parameterizing the Energy Dissipation Rate in ...,We use a database of direct numerical simula...
309769,A General Superapproximation Result,A general superapproximation result is deriv...
20022,Distributed Simulation and Distributed Inference,Independent samples from an unknown probabil...
...,...,...
526534,Fundamental Wireless Performance of a Building,Over 80% of wireless traffic already takes p...
550228,Fifth-order Z-type weighted essentially non-os...,In this paper we propose the variant Z-type ...
442440,Front propagation of a sexual population with ...,The adaptation of biological species to thei...
560352,Equation-of-motion and Lorentz-invariance rela...,Structure functions of polarized spin-1 hadr...


In [ ]:
test_df

,target_text,source_text
499755,Matching of Fracture Functions for SIDIS in Ta...,In the target fragmentation region of Semi-I...
494077,Explicit decay rate for the Gini index in the ...,We investigate the repeated averaging model ...
360040,Color-Dipole Picture versus Hard Pomeron in De...,For photon virtualities of $Q^2 \gsim 20 {\r...
339236,Second Order parallel tensor on generalized f....,The purpose of the present paper to study a ...
193824,Rigorous Theory of the Thin Vapor Layers Optic...,The theory of the thin vapor layers linear o...
...,...,...
175554,Duckiefloat: a Collision-Tolerant Resource-Con...,There are several challenges for search and ...
512385,Compound Krylov subspace methods for parametri...,"In this work, we propose a reduced basis met..."
557468,Gaussian Processes for Finite Size Extrapolati...,Key to being able to accurately model the pr...
135822,"Covering graphs, magnetic spectral gaps and ap...","In this article, we analyze the spectrum of ..."


In [ ]:
!pip install simplet5

In [ ]:
!pip install torchmetrics==0.6.2

     |████████████████████████████████| 332 kB 5.2 MB/s 
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.7.0
    Uninstalling torchmetrics-0.7.0:
      Successfully uninstalled torchmetrics-0.7.0


In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Global seed set to 42


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=2, max_epochs=5, use_gpu=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
! ( cd outputs; ls )

simplet5-epoch-0-train-loss-2.3988  simplet5-epoch-3-train-loss-1.491
simplet5-epoch-1-train-loss-2.0225  simplet5-epoch-4-train-loss-1.3004
simplet5-epoch-2-train-loss-1.7344


In [ ]:
!mv /content/outputs/simplet5-epoch-4-train-loss-1.3004 /content/drive/MyDrive/Blog_3/latest_runs

In [ ]:
# let's load the trained model from the local output folder for inferencing:
model.load_model("/content/outputs/simplet5-epoch-4-train-loss-1.3395", use_gpu=True)

In [ ]:
# let's see how it performerd:
sample_abstracts = test_df.sample(10)

for i, abstract in sample_abstracts.iterrows():
    print(f"===== Abstract =====")
    print(abstract['source_text'])
    summary= model.predict(abstract['source_text'])[0]
    print(f"\n===== Actual Title =====")
    print(f"{abstract['target_text']}")
    print(f"\n===== Generated Title =====")
    print(f"{summary}")
    print("\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

===== Abstract =====
  In this work we propose a robust methodology to mitigate the undesirable
effects caused by outliers to generate reliable physical models. In this way,
we formulate the inverse problems theory in the context of Kaniadakis
statistical mechanics (or $\kappa$-statistics), in which the classical approach
is a particular case. In this regard, the errors are assumed to be distributed
according to a finite-variance $\kappa$-generalized Gaussian distribution.
Based on the probabilistic maximum-likelihood method we derive a
$\kappa$-objective function associated with the finite-variance
$\kappa$-Gaussian distribution. To demonstrate our proposal's
outlier-resistance, we analyze the robustness properties of the
$\kappa$-objective function with help of the so-called influence function. In
this regard, we discuss the role of the entropic index ($\kappa$) associated
with the Kaniadakis $\kappa$-entropy in the effectiveness in inferring physical
parameters by using strongly noi

In [ ]:
# let's see how it performerd:
sample_abstracts = test_df.sample(10)

for i, abstract in sample_abstracts.iterrows():
    print(f"===== Abstract =====")
    print(abstract['source_text'])
    summary= model.predict(abstract['source_text'])[0]
    print(f"\n===== Actual Title =====")
    print(f"{abstract['target_text']}")
    print(f"\n===== Generated Title =====")
    print(f"{summary}")
    print("\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

===== Abstract =====
  This work describes the task of metaphoric paraphrase generation, in which we
are given a literal sentence and are charged with generating a metaphoric
paraphrase. We propose two different models for this task: a lexical
replacement baseline and a novel sequence to sequence model, 'metaphor
masking', that generates free metaphoric paraphrases. We use crowdsourcing to
evaluate our results, as well as developing an automatic metric for evaluating
metaphoric paraphrases. We show that while the lexical replacement baseline is
capable of producing accurate paraphrases, they often lack metaphoricity, while
our metaphor masking model excels in generating metaphoric sentences while
performing nearly as well with regard to fluency and paraphrase quality.


===== Actual Title =====
Metaphoric Paraphrase Generation

===== Generated Title =====
'Metaphor Masking' as a Replacement Baseline and a Sequence to Sequence Model for Paraphrase Generation

 ++++++++++++++++++++++++++